## Generate store information with empty hourly traffic for the next four weeks for forecasting. 

This notebook is continuing using the example described in [Building and deploying a time series forecast with Hex + Snowflake](https://quickstarts.snowflake.com/guide/hex/index.html#0). This entire example higlights how we can use Snowflake to perform parallel hyperparameter tuning forecasting foot traffic. Please take a look at Chase Romano's article [Parallel Hyperparameter tuning using Snowpark](https://medium.com/snowflake/parallel-hyperparameter-tuning-using-snowpark-53cdec2faf77) for more information.

We will begin by establishing our Snowflake connection and Snowpark session. This demo assumes the user has access to the `SYSADMIN` role and a virtual warehouse named `COMPUTE_WH` exists and is available for usage. 

We're assuming the tables `CALENDAR_INFO` and `HOURLY_TRAFFIC` have already been created and populated with data based on the [1-data-ingestion.ipynb](1-data-ingestion.ipynb) notebook in this repository.

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.types import DateType, StringType
from snowflake.snowpark.functions import (
    cast,
    col,
    current_date,
    to_timestamp,
    lit,
    dateadd,
    hour,
    to_date,
)
import pandas as pd
import os

connection_params = {
    "account": os.environ.get("SNOWFLAKE_ACCOUNT"),
    "user": os.environ.get("SNOWFLAKE_USER"),
    "password": os.environ.get("SNOWFLAKE_PASSWORD"),
    "database": os.environ.get("SNOWFLAKE_DATABASE"),
    "schema": os.environ.get("SNOWFLAKE_SCHEMA"),
    "role": "SYSADMIN",
    "warehouse": "COMPUTE_WH",
}

session = Session.builder.configs(connection_params).create()

### Create a DataFrame with relevant historical data.

In [ ]:
store_hourly_info_df = session.table("HOURLY_TRAFFIC")
store_calendar_info_df = session.table("CALENDAR_INFO")

# Extract date and hour from the time stamp in Hourly traffic
past = store_hourly_info_df.select(
    "TIME_POINTS",
    col("TIME_POINTS").cast(DateType()).alias("DATE"),
    hour(col("TIME_POINTS")).alias("HOUR"),
    "STORE_ID",
    "COLLEGE_TOWN",
    "HOURLY_TRAFFIC",
)

# Join the Calendar info table to the Hourly traffic table
# Filter hour between 7 and 22 since the restaraunts are only open from 7am -> 10pm
past_final = (
    past.join(
        store_calendar_info_df,
        (store_calendar_info_df.col("CALENDAR_DATE") == past.col("DATE")),
        "left",
    )
    .select(
        col("TIME_POINTS"),
        col("HOUR"),
        "STORE_ID",
        "COLLEGE_TOWN",
        "CALENDAR_WEEK_DAY_NBR",
        "CALENDAR_MTH_DAY_NBR",
        "CALENDAR_MTH",
        "CALENDAR_YEAR",
        "HOLIDAY_NAME",
        "HOURLY_TRAFFIC",
    )
    .filter(col("HOUR").between(7, 22))
    .na.fill({"HOLIDAY_NAME": "No Holiday"})
)

past_final.limit(10).toPandas()

### Create a column that has the next 672 hours (28 days) in datetime format.

In [ ]:
df_date = session.range(672).select(dateadd("HOUR", "ID", current_date()).as_("DATE"))

df_date = df_date.with_column("HOUR", hour(df_date["DATE"]))

df_date = df_date.select(to_date(df_date["DATE"]).as_("DATE"), "HOUR").filter(
    col("HOUR").between(7, 22)
)

# Cross join to make sure each store gets a value for the next 4 weeks
df_store = (
    session.table("HOURLY_TRAFFIC")
    .select(
        col("STORE_ID").cast("string").alias("STORE_ID"),
        col("COLLEGE_TOWN").cast("string").alias("COLLEGE_TOWN"),
    )
    .distinct()
)
stores = df_date.cross_join(df_store)

### Add in Calendar Information to create the final future table.

In [ ]:
future_cal = (
    session.table("CALENDAR_INFO")
    .select(
        "CALENDAR_DATE",
        "CALENDAR_WEEK_DAY_NBR",
        "CALENDAR_MTH_DAY_NBR",
        "CALENDAR_MTH",
        "CALENDAR_YEAR",
        "HOLIDAY_NAME",
    )
    .filter(
        (col("CALENDAR_DATE") >= current_date())
        & (col("CALENDAR_DATE") <= current_date() + 28)
    )
)

future_cal = future_cal.na.fill({"HOLIDAY_NAME": "No Holiday"})

# Join store info and calendar data
future_df = stores.join(
    future_cal, stores.col("DATE") == future_cal.col("CALENDAR_DATE"), "right"
)
future_df = future_df.drop("CALENDAR_DATE")

future_df = future_df.withColumn(
    "DATE_HOUR", to_timestamp(dateadd("hour", col("HOUR"), col("DATE")))
)
future_df = future_df.drop("DATE")

future_df = future_df.withColumn("HOURLY_TRAFFIC", lit(0))

future_df = future_df.select(
    "DATE_HOUR",
    "HOUR",
    "STORE_ID",
    "COLLEGE_TOWN",
    "CALENDAR_WEEK_DAY_NBR",
    "CALENDAR_MTH_DAY_NBR",
    "CALENDAR_MTH",
    "CALENDAR_YEAR",
    "HOLIDAY_NAME",
    "HOURLY_TRAFFIC",
)

future_df.limit(5).toPandas()

### Union the historical and future tables together and write the final features table to Snowflake.

In [ ]:
union_df = past_final.union(future_df).select(
    "TIME_POINTS",
    "HOUR",
    cast("STORE_ID", StringType()).alias("STORE_ID"),
    "COLLEGE_TOWN",
    "CALENDAR_WEEK_DAY_NBR",
    "CALENDAR_MTH_DAY_NBR",
    "CALENDAR_MTH",
    "CALENDAR_YEAR",
    "HOLIDAY_NAME",
    "HOURLY_TRAFFIC",
)

union_df.write.save_as_table("MODEL_FEATURES", mode="overwrite")